Code for inverting vorticity to get the streamfunction

In [ ]:
# imports
import numpy as np
import matplotlib.pyplot as plt
import scienceplots 
from numpy.fft import rfft, rfftfreq, irfft 
plt.style.use('science')

from grid_interpolation import *

# Functions 

def compute_laplacian(streamfunction, y, x):
    """
    Computes the Laplacian of a given streamfunction.

    Parameters:
    - streamfunction: np.ndarray, 2D streamfunction field.
    - y: np.ndarray, 2D y-coordinates.
    - x: np.ndarray, 2D x-coordinates.

    Returns:
    - xi: np.ndarray, Laplacian of the streamfunction (vorticity).
    """
    
    N = np.shape(y)[1]  # num cols
    M = np.shape(x)[0]  # num rows
    dx = dy = y[0, 1] - y[0, 0]  # extract grid space step from y array
    
    # 3 point centred difference scheme for the interior points of vorticity
    d_y_2 = (streamfunction[:, 2:N] - 
             2 * streamfunction[:, 1:(N-1)] + 
             streamfunction[:, 0:(N-2)]) / (dy**2)

    d_x_2 = (streamfunction[2:M, 1:(N-1)] - 
             2 * streamfunction[1:(M-1), 1:(N-1)] + 
             streamfunction[0:(M-2), 1:(N-1)]) / (dx**2)

    # boundary conditions for the second derivative with respect to x
    d_x_2_first = (streamfunction[1, 1:(N-1)] - 
                   2*streamfunction[0, 1:(N-1)] + 
                   streamfunction[-1, 1:(N-1)]) / (2 * dx)
    
    d_x_2_last  = (streamfunction[0, 1:(N-1)] - 
                   2*streamfunction[-1, 1:(N-1)] + 
                   streamfunction[-2, 1:(N-1)]) / (2 * dx)

    # Stick the boundary conditions onto the interior values
    d_x_2 = np.vstack((d_x_2_first, d_x_2))
    d_x_2 = np.vstack((d_x_2, d_x_2_last))

    # Combine the second derivatives to calculate the vorticity
    xi = d_x_2 + d_y_2
    
    return xi

def generate_tridiagonal_matrix(N, k, grid_spacing):
    """
    Generates a tridiagonal matrix solver for vorticity inversion in the 
    y-direction.

    Parameters:
    - N: int, size of the matrix.
    - k: float, wavenumber.
    - grid_spacing: float, spacing between grid points. Equivalent to dx or dy.

    Returns:
    - matrix: np.ndarray, tridiagonal matrix.
    """

    # define variables and constants
    dy = grid_spacing
    lambda_ = -(k**2)

    # initiate matrix
    matrix = np.zeros((N,N))
    F = 1/(dy**2)

    # fill the matrix with the correct values by looping through the rows 
    # first, then the columns:
    for i in range(N): 
        for j in range(N): 

            # fill the corners of the matrix with 1:
            if (i == j == 0) or (i == j == N-1): 
                matrix[i,j] = 1 
                
            # fill the rest of the matrix with the correct values    
            else:

                # if the row and column are the same, fill the diagonal 
                # with -2F
                if i == j: 
                    matrix[i,j] = lambda_ - (2*F)

                # if the row and column are next to each other, fill 
                # the matrix with F
                elif (i == j+1 or i == j-1) and (i != 0 and i != N-1):
                    matrix[i,j] = F

    return matrix

# constants
phi_0     = 1e5
f_0       = 1e-4
beta      = 1.6e-11
L_y       = 2.5e5
A         = 0.0125
B         = 1e-3
x_0_psi   = 1.25e7
y_0_psi   = 2.5e6
sigma_psi = 1e6

# set up some interpolation orders for experiments
interpolation_order = 3
test_order          = 1
second_test_order   = 5

# duration of simulation
duration  = 6 * 24 * 60 * 60 # 6 days in seconds
dt        = 60 * 60 # 1 hour in seconds
num_steps = duration/dt # number of timesteps

# common grid spacing
grid_space = 1e5

# adding 1e5 to both limits so that np.arrange() includes the upper limit
upper_x_limit = 2.5e7 + grid_space
upper_y_limit = 5e6 + grid_space

# define mid-range of domain in the y-direction
y_0 = 2.5e6

x = np.arange(0, upper_x_limit, grid_space)
y = np.arange(0, upper_y_limit, grid_space)
y_centre = np.mean(y)

# define 2D X and Y arrays:
Y, X = np.meshgrid(y, x)

# We define the streamfunction as follows:
gaussian_bump = phi_0/f_0 * (1 - (A * np.exp(-((X-(x_0_psi))**2 + 
                                 (Y-y_0_psi)**2)/(2 * (sigma_psi**2)))))

# We also compute the cressman term:
L_2_r = phi_0/(f_0**2)

cressman = - (gaussian_bump / L_2_r)

# And the coriolis term:
f = f_0 + (beta * (Y - y_centre))

# Add all of the terms together to get ebv
xi = compute_laplacian(gaussian_bump, Y, X)
ebv = f[:, 1:-1] + xi + cressman[:, 1:-1]

# take the real fourier transform of the x values
ebv_hat = rfft(xi, axis=0)

# form the RHS solution vector
psi_hat = rfft(gaussian_bump, axis=0)

northern_boundary = psi_hat[:, 0]
southern_boundary = psi_hat[:, -1]

# first we add an axis to both the northern and southern boundary arrays
northern_boundary = northern_boundary[:, np.newaxis]
southern_boundary = southern_boundary[:, np.newaxis]

# then we concatenate the arrays together along the column axis
ebv_hat = np.concatenate((northern_boundary, ebv_hat, southern_boundary), 
                         axis=1)

# calculate the wavenumber corresponding to each 'm' value
k = rfftfreq(np.shape(x)[0], grid_space/(2.0*np.pi))

# save number of columns to a variable for use in the loop
num_columns = np.shape(Y)[1]

# make a copy of ebv_hat since it already has fourier terms in x
psi_new = np.copy(ebv_hat)

# Loop through the number of columns and solve the tridiagonal matrix for each
# y-value:
for i in range(num_columns):
    N = np.shape(ebv_hat)[1]
    matrix = generate_tridiagonal_matrix(N, k[i], grid_space)

    # Solve the system of equations
    psi_new[i, :] = np.linalg.solve(matrix, ebv_hat[i, :])

# Inverse Fourier transform the psi_hat_array to get the streamfunction
psi_new = irfft(psi_new, axis=0, n=np.shape(x)[0])


# Visualise the inverted streamfunction against the original 
fig, ax = plt.subplots(2, 1, figsize=(7,5), dpi=200)

for axis in ax:
    axis.set_xlabel('X-Gridpoints')
    axis.set_ylabel('Y-Gridpoints')

ax[0].pcolormesh(psi_new.T) 
ax[1].pcolormesh(gaussian_bump.T)
ax[0].set_aspect('equal')
ax[1].set_aspect('equal')
ax[0].set_title('$\psi_{inverted}$')
ax[1].set_title('$\psi_{gaussian}$')

fig.subplots_adjust(hspace=0.1)  
plt.show()

# look at the difference between the two plots
difference = (psi_new - gaussian_bump)

plt.figure(figsize=(7,2), dpi=200)
plt.axes(aspect='equal')
plt.pcolormesh(difference.T)
plt.ylabel('Y-Gridpoints (m)')
plt.xlabel('X-Gridpoints (m)')
plt.title('Difference between $\psi_{inverted}$ and $\psi_{gaussian}$')
plt.colorbar()

# Calculate the RMSE between psi_new and the original psi
diff_squared = difference**2

# mean squared error
mse = np.mean(diff_squared)

# root mean squared error
rmse = np.sqrt(mse)

print(rmse)